In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class GATES(nn.Module):
    def __init__(self,hidden_dims,A,X,S,R,lambda_=0.01,num_heads=[6,4,2,1]):
        super().__init__()
        self.lambda_ = lambda_
        self.n_layers = len(hidden_dims)-1
        self.skip_proj = nn.ModuleList([nn.Linear(hidden_dims[i],hidden_dims[i+1])for i in range(self.n_layers)])
        self.W,self.v = self.define_weights(hidden_dims,num_heads)
        self.C={}
        self.num_heads = num_heads
        self.threshold = 0.01
        self.A = A
        self.X = X
        self.S = S
        self.R = R
        self.apply(self._init_weights)
    #前向传播
    def forward(self,A,X,R,S):

        #编码
        H = X
        entropy_loss = 0.0 # 注意力熵正则化项
        for layer in range(self.n_layers):
            H,entropy_loss = self.encoder(A, H, layer)
        #最终节点表征
        self.H = H

        #解码
        for layer in range(self.n_layers-1,-1,-1):
            H = self.decoder(H,layer)
        X_ = H

        #节点得重构损失
        features_loss = torch.sqrt(torch.sum((X-X_)**2))

        #图结构得重构损失
        S_emb = self.H[S]
        R_emb = self.H[R]
        structure_loss = -torch.log(torch.sigmoid(torch.sum(S_emb*R_emb,dim=-1))).sum()

        #总损失
        self.loss = features_loss + self.lambda_*structure_loss + entropy_loss*0.1

        return self.loss,self.H,self.C
    #编码器
    def encoder(self,A,H,layer):
        X = H
        H = nn.LeakyReLU(0.2)(H)
        H = torch.matmul(H,self.W[layer].weight.t())

        #多头注意力
        C,entropy = self.graph_attention_layer(A,H,self.v[layer])
        self.C[layer] = C

        #残差连接
        if layer == 0:
            self.skip = H
        else:
            #当维度变化时投影
            if H.shape[1] != self.skip.shape[1]:
                projected_skip = self.skip_proj[layer](self.skip)
                H = H+projected_skip
            else:
                H = H + self.skip
        return torch.matmul(C,X.T),entropy
    #解码器
    def decoder(self,H,layer):
        H = torch.matmul(H,self.W[layer].weight)
        H = nn.BatchNorm1d(H.size(1))(H)  # 添加 BatchNorm
        H = nn.ReLU()(H)
        return torch.matmul(self.C[layer],H)
    
    def define_weights(self,hidden_dims,num_heads):
        #权重定义（支持不同层不同头数）
        W = nn.ModuleList()
        V = nn.ModuleList()

        for i in range(self.n_layers):
            in_dim = hidden_dims[i]*num_heads[i] if i > 0 else hidden_dims[i]
            out_dim = hidden_dims[i+1]

            #投影矩阵
            W.append(nn.Linear(in_dim,out_dim))

            #每层注意力参数（每头）
            head_params = nn.ModuleList([nn.Linear(hidden_dims[i],hidden_dims[i+1],bias=False)for _ in range(num_heads[i])])
            V.append(head_params)
        return W,V
    def graph_attention_layer(self,A,M,v_layer):
        '''
        全连接图注意力层（带阈值剪枝）
        参数：
        M:节点特征矩阵[N,d]
        v:注意力参数列表，包含两个可学习向量[v1,v2]每个形状[d,1]
        layer:层编号
        threshold:注意力权重剪枝阈值(默认0.1)
        '''
        num_heads = len(v_layer)
        head_outputs = []
        total_entroy = 0.0

        for head in range(num_heads):
            # 单头注意力计算
            f = torch.matmul(M,v_layer[head].weight)
            logits = f

            logits = logits/(M.size(1)**0.5)
            attentions = F.softmax(logits,dim=1)
            entropy = -torch.sum(attentions*attentions*torch.log(attentions+1e-10))/(attentions.size(0)*attentions.size(1))
            total_entroy += entropy

            head_outputs.append(attentions)
        
        #合并多头结果
        combined_attentions = torch.stack(head_outputs,dim=0).mean(dim=0)
        return combined_attentions,total_entroy/num_heads
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        elif isinstance(m, nn.ModuleList):  # 针对注意力参数的特殊初始化
            for sub_module in m:
                if isinstance(sub_module, nn.Linear):
                    nn.init.normal_(sub_module.weight, mean=0, std=0.01)



In [2]:
import pandas as pd
import numpy as np
#数据归一化
df = pd.read_excel('standardized_file.xlsx')
for column in df.columns:
    df[column] = (df[column]-df[column].mean())/df[column].std()
df = df.drop(0,axis=1)
data_3d = df.values.reshape(296,22,5)

In [43]:
num_cities = 292
num_nodes = 22
num_features = 5
hidden_dims = [5,16,32,16,5]
lambda_ = 0.3
A = torch.ones(num_nodes,num_nodes,device='cpu')
S,R = np.where(A==1)
embeddings = []


In [117]:
for i in range(num_cities):
    x = torch.from_numpy(data_3d[i]).float()
    model = GATE(hidden_dims,A,x,S,R)
    optimizer = torch.optim.Adam(model.parameters(),lr=0.0002)
    for epoch in range(200):
        
        optimizer.zero_grad()
        loss ,embedding,attention = model(A,x,R,S)
        #print(embedding)
        loss.backward()
        # 在训练循环中添加梯度检查
        # for name, param in model.named_parameters():
        #     if param.grad is None:
        #         print(f"参数 {name} 无梯度")
        #     else:
        #         print(f"参数 {param.shape} 梯度范数: {param.grad.norm().item():.4f},是否可训练{param.requires_grad}")

        optimizer.step()

        if epoch % 20 == 0:
            print(f"Epoch{epoch}:Loss={loss.item():.4f}")
    embeddings.append(embedding)

Epoch0:Loss=73.3026
Epoch20:Loss=58.3728
Epoch40:Loss=57.7111
Epoch60:Loss=58.7125
Epoch80:Loss=57.1865
Epoch100:Loss=57.1565
Epoch120:Loss=56.8670
Epoch140:Loss=56.8652
Epoch160:Loss=58.8043
Epoch180:Loss=56.7726
Epoch0:Loss=39.4928
Epoch20:Loss=29.8621
Epoch40:Loss=25.2030
Epoch60:Loss=29.9943
Epoch80:Loss=28.3375
Epoch100:Loss=28.4318
Epoch120:Loss=26.6754
Epoch140:Loss=31.9385
Epoch160:Loss=22.8927
Epoch180:Loss=21.1126
Epoch0:Loss=28.5567
Epoch20:Loss=27.1948
Epoch40:Loss=19.9365
Epoch60:Loss=20.1987
Epoch80:Loss=12.2330
Epoch100:Loss=24.3314
Epoch120:Loss=14.3721
Epoch140:Loss=14.1623
Epoch160:Loss=11.9638
Epoch180:Loss=11.8913
Epoch0:Loss=35.7577
Epoch20:Loss=35.1664
Epoch40:Loss=31.1825
Epoch60:Loss=20.8563
Epoch80:Loss=19.5978
Epoch100:Loss=30.5878
Epoch120:Loss=20.8052
Epoch140:Loss=19.1375
Epoch160:Loss=19.1144
Epoch180:Loss=19.7122
Epoch0:Loss=20.1925
Epoch20:Loss=13.4991
Epoch40:Loss=8.4519
Epoch60:Loss=8.5862
Epoch80:Loss=7.1044
Epoch100:Loss=4.7539
Epoch120:Loss=4.9791
E

KeyboardInterrupt: 

In [ ]:
class GATE(nn.Module):
    def __init__(self,hidden_dims,A,X,S,R,lambda_=0.05):
        super().__init__()
        self.lambda_ = lambda_
        self.n_layers = len(hidden_dims)-1
        self.W,self.v = self.define_weights(hidden_dims)
        self.C={}
        self.threshold = 0.01
        self.A = A
        self.X = X
        self.S = S
        self.R = R
        self.res_linear = nn.ModuleList([nn.Linear(hidden_dims[i],hidden_dims[i+1])for i in range(len(hidden_dims)-1)])
        self.apply(self._init_weights)
    #前向传播
    def forward(self,A,X,R,S):

        #编码
        H = X
        for layer in range(self.n_layers):
            H = self.encoder(A, H, layer)
        #最终节点表征
        self.H = H

        #解码
        for layer in range(self.n_layers-1,-1,-1):
            H = self.decoder(H,layer)
        X_ = H

        #节点得重构损失
        features_loss = torch.sqrt(torch.sum((X-X_)**2))

        #图结构得重构损失
        S_emb = self.H[S]
        R_emb = self.H[R]
        structure_loss = -torch.log(torch.sigmoid(torch.sum(S_emb*R_emb,dim=-1))).sum()

        #总损失
        self.loss = features_loss + self.lambda_*structure_loss

        return self.loss,self.H,self.C
    def encoder(self,A,H,layer):
        identity = H
        H = nn.LeakyReLU(0.2)(H)
        H = torch.matmul(H,self.W[layer].weight.t())
        H = nn.BatchNorm1d(H.size(1))(H) 
        self.C[layer]= self.graph_attention_layer(A,H,self.v[layer],layer,self.threshold)
        H = torch.matmul(self.C[layer],H)
        if H.shape[-1] == identity.shape[-1]:
            H = H+identity
        else:
            H = H+self.res_linear[layer](identity)
        return H 
    #解码器
    def decoder(self,H,layer):
        H = torch.matmul(H,self.W[layer].weight)
        H = nn.BatchNorm1d(H.size(1))(H)   #添加 BatchNorm
        H = nn.ReLU()(H)
        return torch.matmul(self.C[layer],H)
    
    def define_weights(self,hidden_dims):
        W = nn.ModuleList(nn.Linear(hidden_dims[i],hidden_dims[i+1])for i in range(self.n_layers))
        Ws_att = nn.ModuleList([nn.ModuleList([nn.Linear(hidden_dims[i+1],1,bias=False),nn.Linear(hidden_dims[i+1],1,bias=False)])for i in range(self.n_layers)])
        return W,Ws_att
    def graph_attention_layer(self,A,M,v,layer,threshold):
        '''
        全连接图注意力层（带阈值剪枝）
        参数：
        M:节点特征矩阵N,d
        v:注意力参数列表,包含两个可学习向量v1,v2每个形状d,1
        layer:层编号
        '''
        #启用异常检测，监控反向传播
        with torch.autograd.set_detect_anomaly(True):
            #计算全连接注意力分数
            f1 = torch.matmul(M,v[0].weight.t())
            f2 = torch.matmul(M,v[1].weight.t())

            #生成全连接分数矩阵N,N
            logits = f1 + f2.T
            logits = F.dropout(logits,p=0.1,training=self.training)
            logits = logits / torch.max(torch.abs(logits))   #归一化
            #激活函数
            unnormalized_attentions = torch.exp(logits)
            
        #     #行方向归一化
            attentions = F.softmax(unnormalized_attentions,dim=1)
         
            #print(attentions.mean(), attentions.min(), attentions.max())
        #     #阈值剪枝
            return attentions
    
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        elif isinstance(m, nn.ModuleList):   #针对注意力参数的特殊初始化
            for sub_module in m:
                if isinstance(sub_module, nn.Linear):
                    nn.init.normal_(sub_module.weight, mean=0, std=0.01)

In [55]:
data = torch.stack(embeddings)
print(data.shape)
numpy_data = data.detach().numpy()
numpy_data = numpy_data.reshape(292,110)
print(numpy_data.shape)
df = pd.DataFrame(numpy_data)
df.to_excel('embedding.xlsx')

torch.Size([292, 22, 5])
(292, 110)


In [67]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

In [ ]:
embedding_s = torch.tensor(numpy_data,dtype=torch.float32)
model = GATEWithClustering(embedding_s)
cluster_labels = model.cluster_cities()
print(cluster_labels)

In [87]:
class GATEWithClustering():
    def __init__(self,embeddings):
        super().__init__()
        self.cluster_labels = None
        self.cluster_centers = None
        self.lambda_ = 0.01
        self.embeddings = embeddings
    def cluster_cities(self,n_clusters = 3,visualize = True):
        '''
        城市韧性等级聚类(5类)
        参数：
          n_cluters:聚类数量（韧性等级数）
          visualize:是否进行降维可视化
        '''
        #获取节点嵌入
        X_emb =self.embeddings

        #使用KMeans聚类
        kmeans = KMeans(n_clusters=n_clusters,random_state = 42,n_init=10)
        self.cluster_labels = kmeans.fit_predict(X_emb)
        self.cluster_centers = kmeans.cluster_centers_
    
        #聚类结果分析
        self._analyze_clusters()

        #可视化嵌入空间
        if visualize and X_emb.shape[1]>2:
            self._visualize_embeddings(X_emb)

        return self.cluster_labels
    
    def _analyze_clusters(self):
        """分析聚类结果"""
        unique,counts = np.unique(self.cluster_labels,return_counts=True)
        print("\n城市韧性等级分布")
        for cls,cnt in zip(unique,counts):
            print(f"等级{cls+1}:{cnt}个城市(占比{cnt/len(self.cluster_labels):.1%})")
        #计算轮廓系数
        from sklearn.metrics import silhouette_score
        sil_score = silhouette_score(self.embeddings.detach().cpu().numpy(),self.cluster_labels)
        print(f"\n轮廓系数:{sil_score:.3f}(越接近1表示聚类效果越好)")

    def _visualize_embeddings(self,X_emb,perplexity=2):
        """使用t-SNE降维可视化"""
        tsne = TSNE(n_components=2,perplexity=perplexity,random_state=42)
        X_tsne = tsne.fit_transform(X_emb)

        plt.figure(figsize=(10,8))
        scatter = plt.scatter(X_tsne[:,0],X_tsne[:,1],
                              c = self.cluster_labels,
                              cmap='viridis',
                              alpha=0.7,
                              edgecolors='w')
        #添加图例和标签
        plt.colorbar(scatter,ticks=range(5),label='韧性等级')
        plt.title("城市韧性嵌入空间可视化(t-SNE)")
        plt.xlabel("t-SNE 1")
        plt.ylabel("t-SNE 2")

        #标记聚类中心
        centers_tsne = tsne.fit_transform(self.cluster_centers)
        plt.scatter(centers_tsne[:,0],centers_tsne[:,1],
                    c='red',marker='X',s=200,
                    edgecolors='k',linewidths=1.5,
                    label='等级中心')
        plt.legend()
        plt.show()